In [74]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from datetime import datetime
import numpy as np
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from scipy.stats import randint as sp_randint
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestClassifier

In [75]:
train_data = pd.read_csv("data/train.csv")
test_data = pd.read_csv("data/test.csv")
val_data = pd.read_csv("data/val.csv")

In [76]:
labels_train = train_data["amount_spent_per_room_night_scaled"]
labels_test = test_data["amount_spent_per_room_night_scaled"]
labels_val = val_data["amount_spent_per_room_night_scaled"]

In [77]:
def convert_datetime(df):
    df["checkin_date"] = pd.to_datetime(df['checkin_date'])
    df["checkout_date"] = pd.to_datetime(df['checkout_date'])
    df["booking_date"] = pd.to_datetime(df['booking_date'])
    return df

train_data= convert_datetime(train_data)
test_data= convert_datetime(test_data)
val_data= convert_datetime(val_data)

C:\Users\Kush\AppData\Local\Temp\ipykernel_44684\2227079341.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["checkin_date"] = pd.to_datetime(df['checkin_date'])
C:\Users\Kush\AppData\Local\Temp\ipykernel_44684\2227079341.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["checkout_date"] = pd.to_datetime(df['checkout_date'])
C:\Users\Kush\AppData\Local\Temp\ipykernel_44684\2227079341.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["booking_date"] = pd.to_datetime(df['booking_date'])
C:\Users\Kush\AppData\Local\Temp\ipykernel_44684\2227079341.py:2: UserWarn

In [78]:
temp_train = train_data.copy()
temp_test = test_data.copy()
temp_val = val_data.copy()

In [79]:
def calssification(labels):
    label_cat = []
    for i in labels:
        if i < 5:
            label_cat.append("less than 5")
        elif 5 <= i < 6:
            label_cat.append("between 5 and 6")
        elif 6 <= i < 7:
            label_cat.append("between 6 and 7")
        elif 7 <= i < 8:
            label_cat.append("between 7 and 8")
        elif 8 <= i < 9:
            label_cat.append("between 8 and 9")
        else:
            label_cat.append("greater than or equal to 9")
    return label_cat

In [80]:
temp_train["amount_range"] = calssification(labels_train)
temp_test["amount_range"] = calssification(labels_test)
temp_val["amount_range"] = calssification(labels_val)

In [81]:
temp_test.T

,0,1,2,3,4,5,6,7,8,9,...,11990,11991,11992,11993,11994,11995,11996,11997,11998,11999
reservation_id,f10db4056bd0fd104d07887e3f2610442e1ed4460a15a0...,10d7445a92746ef1d244f2d5c85ba4a7251c3aae9f91db...,68cfd1b097a5a14b3aeea85c54fa7f8d402a367167906a...,fbae42db0552db8b359dab6d6485e24eb07e9c8549f7a6...,348c49082448e0c51c4f3460ab8c7ace24c0f958da2c21...,430acd855c61e68db049fd26a01505c5c19c04ff7af16f...,456a76a0a16d3f885445c6953b7e1c610fbc928c07f816...,56ea6c36f9ecb5a890bdb29e3189a1031925e58b5f8fb8...,41b68e2f1a0cb96156d9713bd4f6d30a145e64b93b2467...,a9801776f38cf8768279873234aaaadfa5e8a5a8441c81...,...,99f29f5bdc5353f6e81ed9f2a9ff629230666b9facbe4d...,037093f64d1ca9ee7980557756bd63bb879e43c20f79e0...,487798e33ce1b82c6c9f54b80d9bbf91991cd44a5c8e28...,9d7c85ad643745910b3f3ed709c6974c2b3c155244d859...,74d4112cc5ce271c25d7b304be825ccb85e47bb7aa3c87...,e7f4aaa24919491c2d7ba3aca9a154b0fc8ca8f71df7bf...,636a1a2f6f356f753effa0084a5fdaca48e0d795acb50b...,0bb6e43be5e3869805caa28b78098df9298ca7e879792c...,09e4d4de4f1928bb1181570b7f283c6c6d81ef9316b0d5...,c6ad459c583abb12da0364d6c5475fb05be18aa9289ad8...
booking_date,2017-03-11 00:00:00,2018-12-07 00:00:00,2016-05-20 00:00:00,2016-12-16 00:00:00,2018-03-09 00:00:00,2017-02-02 00:00:00,2016-04-04 00:00:00,2015-04-29 00:00:00,2016-12-12 00:00:00,2017-01-18 00:00:00,...,2016-07-15 00:00:00,2017-04-14 00:00:00,2017-10-06 00:00:00,2017-07-10 00:00:00,2017-05-20 00:00:00,2016-03-17 00:00:00,2017-05-09 00:00:00,2017-10-19 00:00:00,2016-02-01 00:00:00,2018-04-23 00:00:00
checkin_date,2017-12-13 00:00:00,2018-07-27 00:00:00,2016-06-07 00:00:00,2017-09-01 00:00:00,2018-12-17 00:00:00,2017-05-31 00:00:00,2016-11-05 00:00:00,2015-12-07 00:00:00,2017-12-01 00:00:00,2017-11-03 00:00:00,...,2016-08-30 00:00:00,2017-05-22 00:00:00,2017-07-07 00:00:00,2017-10-27 00:00:00,2017-06-19 00:00:00,2016-05-14 00:00:00,2017-09-14 00:00:00,2017-12-19 00:00:00,2016-01-23 00:00:00,2018-04-23 00:00:00
checkout_date,2017-12-15 00:00:00,2018-07-29 00:00:00,2016-08-07 00:00:00,2017-01-13 00:00:00,2018-12-21 00:00:00,2017-04-06 00:00:00,2016-05-13 00:00:00,2015-07-16 00:00:00,2017-01-17 00:00:00,2017-12-03 00:00:00,...,2016-01-09 00:00:00,2017-05-26 00:00:00,2017-09-07 00:00:00,2017-10-30 00:00:00,2017-06-22 00:00:00,2016-05-17 00:00:00,2017-09-15 00:00:00,2017-12-23 00:00:00,2016-01-26 00:00:00,2018-04-24 00:00:00
channel_code,3,2,1,1,1,1,3,1,3,2,...,3,3,3,1,3,1,1,1,1,3
main_product_code,3,2,1,2,2,1,1,2,1,2,...,4,2,1,2,3,2,3,2,2,1
numberofadults,3,6,10,2,2,2,5,5,4,4,...,8,4,4,6,4,2,4,2,2,3
numberofchildren,0,3,0,2,0,2,0,0,0,0,...,2,0,0,0,0,0,0,0,2,0
persontravellingid,45,45,45,45,45,45,46,47,45,4752,...,45,46,46,45,45,45,45,45,45,45
resort_region_code,1,2,3,2,2,2,2,2,2,3,...,1,3,2,3,2,2,1,3,1,2


In [82]:
def formation_based_on_date(df):

    # Check in week, month and year 
    df["checkin_week"] = df['checkin_date'].dt.isocalendar().week
    df["checkin_month"] = df['checkin_date'].dt.month
    df["checkin_year"] = df['checkin_date'].dt.year

    # Checkout week and month 
    df["checkout_week"] = df['checkout_date'].dt.isocalendar().week
    df["checkout_month"] = df['checkout_date'].dt.month

    # Advance booking and days stayed
    df["advance_booking"] = (df['checkin_date'] - df["booking_date"]).dt.days
    df["days_stayed"] = (df['checkout_date'] - df['checkin_date']).dt.days

    # To count total weekdays
    df["weekdays_stayed"] = np.busday_count(df['checkin_date'].values.astype('datetime64[D]'), 
                                                df['checkout_date'].values.astype('datetime64[D]'))

    # To count total weekends
    df["weekends_stayed"] = df["days_stayed"] - df["weekdays_stayed"]

    # Total dropped days
    df["dropped_days"] = df["roomnights"] - df["days_stayed"]

    # Total person days
    df["total_pax_days"] = df['days_stayed'] * df['total_pax']


    return df

In [83]:
temp_train = formation_based_on_date(temp_train)
temp_test = formation_based_on_date(temp_test)
temp_val = formation_based_on_date(temp_val)

In [84]:
temp_train.T

,0,1,2,3,4,5,6,7,8,9,...,38390,38391,38392,38393,38394,38395,38396,38397,38398,38399
reservation_id,af59a353354e1f770f74dfded465188551eecfa178fa14...,f181967e96e4384c4602e83972cf599be88086c147fafc...,5bc545fffdf66fd7b6da6e63711df565721542b145d5da...,ae726f5288bcd1a4c94d717ec767fbd73413ef24bcae92...,22591667173e9ba038376579f5750965a3a439304985ce...,22a2111d19d4abfbfab8a65cc1f81e52db09e096a6511a...,91815eb6f8859e680cbc786d6f018567383d3fd8162e7e...,40cbf74d4e2457f46029b45715b3812349964172bf7407...,9d72d3972f0a334ca8aaebb89025e20f746ab6b8bf97ea...,45172b19870e988ce09f83f4c0a79a100cacff6f06d88e...,...,f06bb907fdce3cfe250c2e2cfddb5a79639309e9ae5553...,57885d67c8b71b62bfbe5bd30488fe04dc673e52b18b44...,6b3a004512d7d2e702562afcaf9c45257e4b813aabd15d...,036ef2fad7c0ae76f070b68cc51c84b933b9e31f9452cf...,9364d569e9149dfec61fd29218edd4fe318c90f894cf50...,05a9ddd2d3b3865aee948409bec362186e219a32f0cc1d...,3e4cd47769c1e3b6b309fe9a3b6ef5844d75ecd328af59...,50ff4b7deb119747684dfa13e09a340b93a314c3a2b34a...,5a396595b2ed3eeb4448a240c230a0d9c718813a91776c...,a9b5a33fb9052315259763e6563cba6826ff008eb3f1b0...
booking_date,2018-04-26 00:00:00,2018-03-10 00:00:00,2017-01-01 00:00:00,2017-05-30 00:00:00,2015-08-01 00:00:00,2014-10-16 00:00:00,2017-04-02 00:00:00,2017-09-09 00:00:00,2017-01-19 00:00:00,2017-12-09 00:00:00,...,2018-04-01 00:00:00,2017-03-23 00:00:00,2017-05-22 00:00:00,2018-07-21 00:00:00,2015-07-30 00:00:00,2015-06-05 00:00:00,2018-12-02 00:00:00,2018-06-09 00:00:00,2016-01-08 00:00:00,2016-12-18 00:00:00
checkin_date,2018-01-06 00:00:00,2018-10-19 00:00:00,2017-11-04 00:00:00,2017-05-31 00:00:00,2015-03-19 00:00:00,2015-01-01 00:00:00,2017-04-30 00:00:00,2017-11-26 00:00:00,2017-03-04 00:00:00,2017-12-10 00:00:00,...,2018-04-20 00:00:00,2017-05-13 00:00:00,2017-05-22 00:00:00,2018-06-08 00:00:00,2015-04-09 00:00:00,2015-11-05 00:00:00,2018-06-04 00:00:00,2018-04-12 00:00:00,2016-11-13 00:00:00,2017-05-01 00:00:00
checkout_date,2018-05-06 00:00:00,2018-10-20 00:00:00,2017-04-15 00:00:00,2017-01-06 00:00:00,2015-03-21 00:00:00,2015-04-01 00:00:00,2017-05-05 00:00:00,2017-01-12 00:00:00,2017-05-04 00:00:00,2017-10-15 00:00:00,...,2018-04-22 00:00:00,2017-05-15 00:00:00,2017-05-23 00:00:00,2018-09-08 00:00:00,2015-06-09 00:00:00,2015-05-13 00:00:00,2018-07-04 00:00:00,2018-06-12 00:00:00,2016-11-16 00:00:00,2017-08-01 00:00:00
channel_code,2,1,1,1,3,1,1,1,1,2,...,1,1,3,1,3,3,2,1,1,1
main_product_code,2,1,1,2,3,1,2,1,2,1,...,2,2,2,2,4,2,3,3,2,1
numberofadults,4,3,4,2,3,4,2,6,2,2,...,2,5,3,2,3,4,2,2,4,2
numberofchildren,4,0,0,0,0,2,2,0,0,1,...,2,1,0,0,0,0,0,0,0,0
persontravellingid,4752,45,45,45,45,45,45,45,47,45,...,45,45,45,45,46,45,45,4752,45,45
resort_region_code,1,3,1,1,1,2,1,1,2,3,...,3,2,1,1,1,3,1,2,2,3


In [85]:
def formation_based_on_groupby(df):
    
    calc_mean = df.groupby(['resort_id'], axis=0).agg(
        {"total_pax":"mean","days_stayed":"mean","advance_booking":"mean","total_pax_days":"mean"}).reset_index()
    calc_mean.columns = ['resort_id','totalpax_mean',"days_stayed_resmean","advance_booking_resmean","totpaxdays_resmean"]
    df = df.merge(calc_mean,on=['resort_id'],how='left')

    calc_mean = df.groupby(['memberid'], axis=0).agg(
        {"total_pax":"mean","days_stayed":"mean","advance_booking":"mean","reservation_id":"count",
         "roomnights":"mean","numberofadults":"mean","numberofchildren":"mean","weekends_stayed":"mean", "weekdays_stayed":"mean",
         "total_pax_days":"mean"}).reset_index()
    calc_mean.columns = ['memberid','totalpax_memmean',"days_stayed_memmean","advance_booking_memmean",
                         "res_memcnt","roomnights_memmean","adults_memmean","child_memmean","weekends_memmean","weekdays_memmean",
                         "totpaxdays_memmean"]
    df = df.merge(calc_mean,on=['memberid'],how='left')

    calc_mean = df.groupby(['memberid'], axis=0).agg(
        {"days_stayed":"sum","advance_booking":"sum","total_pax":"sum","numberofadults":"sum","numberofchildren":"sum",
         "weekends_stayed":"sum","total_pax_days":"sum"}).reset_index()
    calc_mean.columns = ['memberid',"days_stayed_memsum","advance_booking_memsum","total_pax_memsum","adults_memsum","child_memsum",
                         "weekend_memsum","totpaxdays_memsum"]
    df = df.merge(calc_mean,on=['memberid'],how='left')

    calc_mean = df.groupby(['memberid','resort_id'], axis=0).agg(
        {"total_pax":"mean","days_stayed":"mean","roomnights":"mean","reservation_id":"count","total_pax_days":"mean"}).reset_index()
    calc_mean.columns = ['memberid','resort_id','totalpax_memresmean',"days_stayed_memresmean","roomnights_memresmean",
                         "book_memrescnt","totpaxdays_memresmean"]
    df = df.merge(calc_mean,on=['memberid','resort_id'],how='left')

    df["passengers_dropped"] = df["total_pax"] - (df["numberofadults"] + df["numberofchildren"])
    
    drop = ['checkin_date','checkout_date','booking_date','memberid']
    df = df.drop(drop, axis=1)
    
    df = df.dropna()

    return df

In [86]:
temp_train = formation_based_on_groupby(temp_train)
temp_test = formation_based_on_groupby(temp_test)
temp_val = formation_based_on_groupby(temp_val)

C:\Users\Kush\AppData\Local\Temp\ipykernel_44684\1244930402.py:3: FutureWarning: The 'axis' keyword in DataFrame.groupby is deprecated and will be removed in a future version.
  calc_mean = df.groupby(['resort_id'], axis=0).agg(
C:\Users\Kush\AppData\Local\Temp\ipykernel_44684\1244930402.py:8: FutureWarning: The 'axis' keyword in DataFrame.groupby is deprecated and will be removed in a future version.
  calc_mean = df.groupby(['memberid'], axis=0).agg(
C:\Users\Kush\AppData\Local\Temp\ipykernel_44684\1244930402.py:17: FutureWarning: The 'axis' keyword in DataFrame.groupby is deprecated and will be removed in a future version.
  calc_mean = df.groupby(['memberid'], axis=0).agg(
C:\Users\Kush\AppData\Local\Temp\ipykernel_44684\1244930402.py:24: FutureWarning: The 'axis' keyword in DataFrame.groupby is deprecated and will be removed in a future version.
  calc_mean = df.groupby(['memberid','resort_id'], axis=0).agg(
C:\Users\Kush\AppData\Local\Temp\ipykernel_44684\1244930402.py:3: FutureW

In [87]:
temp_train.isna().sum().any()

False

In [88]:
categorical_vars =temp_train.select_dtypes(include=['object']).columns.tolist()
categorical_vars

['reservation_id',
 'member_age_buckets',
 'cluster_code',
 'reservationstatusid_code',
 'resort_id',
 'amount_range']

In [89]:
temp_train["amount_range"].unique()

array(['between 7 and 8', 'between 8 and 9', 'less than 5',
       'greater than or equal to 9', 'between 5 and 6', 'between 6 and 7'],
      dtype=object)

In [90]:
def object_to_category(df):
    
    categorical_vars =temp_train.select_dtypes(include=['object']).columns.tolist()
    
    for i in categorical_vars:
        if i not in ['reservation_id','amount_range']:
            lbl = LabelEncoder()
            lbl.fit(list(df[i].values))
            df[i] = lbl.transform(list(df[i].values))

    for var in categorical_vars:
        df[var] = df[var].astype('category')
    
    return df

In [91]:
temp_test = object_to_category(temp_test)
temp_train = object_to_category(temp_train)
temp_val = object_to_category(temp_val)

In [92]:
temp_train.T

,0,1,2,3,4,5,6,7,8,9,...,38390,38391,38392,38393,38394,38395,38396,38397,38398,38399
reservation_id,af59a353354e1f770f74dfded465188551eecfa178fa14...,f181967e96e4384c4602e83972cf599be88086c147fafc...,5bc545fffdf66fd7b6da6e63711df565721542b145d5da...,ae726f5288bcd1a4c94d717ec767fbd73413ef24bcae92...,22591667173e9ba038376579f5750965a3a439304985ce...,22a2111d19d4abfbfab8a65cc1f81e52db09e096a6511a...,91815eb6f8859e680cbc786d6f018567383d3fd8162e7e...,40cbf74d4e2457f46029b45715b3812349964172bf7407...,9d72d3972f0a334ca8aaebb89025e20f746ab6b8bf97ea...,45172b19870e988ce09f83f4c0a79a100cacff6f06d88e...,...,f06bb907fdce3cfe250c2e2cfddb5a79639309e9ae5553...,57885d67c8b71b62bfbe5bd30488fe04dc673e52b18b44...,6b3a004512d7d2e702562afcaf9c45257e4b813aabd15d...,036ef2fad7c0ae76f070b68cc51c84b933b9e31f9452cf...,9364d569e9149dfec61fd29218edd4fe318c90f894cf50...,05a9ddd2d3b3865aee948409bec362186e219a32f0cc1d...,3e4cd47769c1e3b6b309fe9a3b6ef5844d75ecd328af59...,50ff4b7deb119747684dfa13e09a340b93a314c3a2b34a...,5a396595b2ed3eeb4448a240c230a0d9c718813a91776c...,a9b5a33fb9052315259763e6563cba6826ff008eb3f1b0...
channel_code,2,1,1,1,3,1,1,1,1,2,...,1,1,3,1,3,3,2,1,1,1
main_product_code,2,1,1,2,3,1,2,1,2,1,...,2,2,2,2,4,2,3,3,2,1
numberofadults,4,3,4,2,3,4,2,6,2,2,...,2,5,3,2,3,4,2,2,4,2
numberofchildren,4,0,0,0,0,2,2,0,0,1,...,2,1,0,0,0,0,0,0,0,0
persontravellingid,4752,45,45,45,45,45,45,45,47,45,...,45,45,45,45,46,45,45,4752,45,45
resort_region_code,1,3,1,1,1,2,1,1,2,3,...,3,2,1,1,1,3,1,2,2,3
resort_type_code,1,4,5,1,1,4,1,1,4,3,...,3,1,1,1,1,3,1,7,0,1
room_type_booked_code,3,3,2,2,3,3,3,2,3,4,...,2,1,3,3,3,2,3,4,4,3
roomnights,8,1,4,1,2,6,5,14,2,6,...,2,3,1,3,2,4,4,2,6,3


In [93]:
def remove_outliers(df):
    df = df.drop(df[df['roomnights'] < 0].index)
    df = df.drop(df[df['advance_booking'] < 0].index)
    df = df.drop(df[df['days_stayed'] < 0].index)
    df = df.drop(df[(df["numberofadults"] + df["numberofchildren"]) == 0].index)
    df = df.drop(df[df['total_pax'] == 0].index)

    return df

In [94]:
temp_test = remove_outliers(temp_test)
temp_train = remove_outliers(temp_train)
temp_val = remove_outliers(temp_val)

In [95]:
X = temp_train.drop(["amount_range","amount_spent_per_room_night_scaled","reservation_id"],axis=1)
Y = temp_train["amount_range"]

In [96]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y, test_size=.2,random_state=42)

In [97]:
from sklearn.metrics import accuracy_score
model = RandomForestClassifier()
model.fit(X_train,Y_train)
# Make predictions on the testing set
y_pred = model.predict(X_test)

# Evaluate model performance
accuracy = accuracy_score(Y_test, y_pred)
print("Accuracy:", accuracy)
print(model.score(X_train,Y_train))
# print(model.score(X_test,Y_test))

Accuracy: 0.44663713250100684
0.9990937012235034


In [98]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
gbm = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, random_state=42)
# Train the model
gbm.fit(X_train, Y_train)

# Make predictions on the testing set
y_pred = gbm.predict(X_test)

# Evaluate model performance
accuracy = accuracy_score(Y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.4679822795006041


In [99]:
from sklearn.preprocessing import LabelEncoder

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Encode the string labels into numerical format
Y = label_encoder.fit_transform(Y)


In [100]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y, test_size=.2,random_state=42)

In [101]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score



# Assuming X_train is your DataFrame and you have a list of categorical feature names
categorical_features = ['member_age_buckets', 'cluster_code', 'reservationstatusid_code', 'resort_id']

# Get the indices of the categorical columns
cat_features_indices = [X_train.columns.get_loc(feature) for feature in categorical_features]

# Now cat_features_indices is a list of indices for the categorical columns


# cat_features_indices = [list(X_train.columns).index( ['member_age_buckets', 'cluster_code', 'reservationstatusid_code','resort_id'])]
catboost_model = CatBoostClassifier(verbose=0, cat_features=cat_features_indices)

# Initialize a dictionary to store trained models
models = {
    'Logistic Regression': LogisticRegression(),
    'Decision Tree': DecisionTreeClassifier(),
    'Random Forest': RandomForestClassifier(),
    'SVM': SVC(),
    'K-Nearest Neighbors': KNeighborsClassifier(),
    # 'XGBoost': XGBClassifier(),
    'Gradient Boosting Machine': GradientBoostingClassifier(),
    'CatBoost': catboost_model,
    'LightGBM': LGBMClassifier()
}

# Train and predict using each model in the dictionary
results = {}
for name, model in models.items():
    # Train the model
    model.fit(X_train, Y_train)
    
    # Make predictions on the testing set
    y_pred = model.predict(X_test)
    
    # Evaluate model performance
    accuracy = accuracy_score(Y_test, y_pred)
    
    # Store results
    results[name] = accuracy

# Print results
for name, accuracy in results.items():
    print(f"{name}: Accuracy = {accuracy:.4f}")


c:\Users\Kush\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001840 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3505
[LightGBM] [Info] Number of data points in the train set: 19861, number of used features: 56
[LightGBM] [Info] Start training from score -3.093008
[LightGBM] [Info] Start training from score -2.028642
[LightGBM] [Info] Start training from score -1.039710
[LightGBM] [Info] Start training from score -1.042705
[LightGBM] [Info] Start training from score -2.368182
[LightGBM] [Info] Start training from score -3.754476
Logistic Regression: Accuracy = 0.3925
Decision Tree: Accuracy = 0.3419
Random Forest: Accuracy = 0.4531
SVM: Accuracy = 0.3739
K-Nearest Neighbors: Accuracy = 0.3399
Gradient Boosting Machine: Accuracy = 0.4682
CatBoost: Accuracy = 0.4672
LightGBM: Accuracy = 0.4678
